This notebook aims on cross-compare and double check accuracy and precision of topic modeling label. </p>
Main steps in this notebook:</p>
1) pull up representative speeches labled by each model about environment;</p>
2) cross-check the results from each model: do they share a lot common? Do all models label the same group of speech as environment-relevant? </p>
3) Mannual check precision and accuracy for randomly selected speeches.</p>
4) Explore collocated-words in interested speeches.</p>
5) merge speeches with speech information (like speakers information) and output environment speeches and data for peer lab member to further investigate

In [ ]:
# code to find topic with Standford keywords
# environment: 3/ 520
# sd 406: https://www.sanders.senate.gov/press-releases/sanders-boxer-to-introduce-major-climate-change-legislation-2/
envir_topic_bert = []
for topicIndex in topics:
    for pair in topics[topicIndex]:
        for i in range(len(pair)):
            for word in environment_keyWord:
                if word == pair[i]:
                    envir_topic_bert.append(topicIndex)
                    print(topicIndex)  # each key words
            

In [29]:
import pandas as pd
fullData = pd.read_csv('test50_labled.csv')
bert_kw_List = [3,2761, 4476, 2343, 4294, 7408, 43,110,115,595,672,849,1311,2643,2805,2855,3184,3878,4148,4690,4821,4887,5892,6154,6159,6228,6532,6675,6820]
lda_kw_List = [9]
nmf_kw_List = [18,19,25]

In [18]:
new_data = pd.read_csv('test50.csv')
fullData = pd.merge(fullData, new_data[['speech_id', 'date']], on='speech_id', how='left')
fullData = pd.merge(fullData, new_data[['speech_id', 'Year']], on='speech_id', how='left')

In [19]:
import numpy as np
import csv
import glob
import nltk
from nltk.corpus import wordnet as wn

import nltk
nltk.download('wordnet')

from collections import Counter
fullDataBack =fullData
fullData['speech'] = fullData['speech'].str.split().dropna()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
def lemmatize_words(word_list):
    if isinstance(word_list, list):
        return [wn.morphy(word) for word in word_list]
    else:
        return []

# Apply the lemmatization function to the 'speech' column
fullData['lemmas'] = fullData['speech'].apply(lemmatize_words)

In [35]:
def convert_to_list(s):
    try:
        # Try to evaluate the string as a Python literal
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        # If there's an error, return an empty list
        return []

# Apply the conversion function to the 'speech' column
fullData['speech'] = fullData['speech'].apply(convert_to_list)

# Filter out rows containing the word 'energy'
filtered_data = fullData[fullData['speech'].apply(lambda x: 'oil' in x)]


In [36]:
filtered_data

""


In [33]:
#fullData = fullData[fullData["speech"].contains('energy')]
#fullData['lemmas'] = [[wn.morphy(word) for word in row] for row in fullData['speech']]
fullData=filtered_data
fullData['lemmas'] = [[word for word in row if word is not None] for row in fullData['lemmas']] # for each word of each row in 'lemmas', keep anything that isn't 'None' type.
#fullData['lemmas'] = [' '.join(row) for row in fullData['lemmas']] # glue the individual list of lemmas back into one string per speech

top_words_per_year_lemmatized = {}

for year in fullData['Year'].unique():
    yearly_lemmas = []
    for lemmas in fullData[fullData['Year'] == year]['lemmas']:
        yearly_lemmas.extend(lemmas)

    # Calculate frequency
    lemma_counts = Counter(yearly_lemmas)

    # Get the 10 most common lemmas
    top_lemmas = lemma_counts.most_common(10)
    top_words_per_year_lemmatized[year] = top_lemmas

top_words_per_year_lemmatized

{}

In [ ]:
fullData['speech_id'].describe()
#1060000000 to 1140000000
filtered_data = fullData[(fullData['speech_id'] >= 1060000000) & (fullData['speech_id'] < 1070000000)]['speech']


In [ ]:
year1_context_count = fullData["speech"].str.split().explode().dropna().value_counts()
year1_context_count[:20]

In [ ]:
pd.set_option('display.max_colwidth', None)
fullData[fullData['Topic']==2343]['speech'].head(3)

In [ ]:
# print out each topic keywords with an example
pd.set_option('display.max_colwidth', None)
for num in bert_kw_List:
    print(num)
    print(fullData[fullData['Topic']==num]['Representation'].head(1))
    print(fullData[fullData['Topic']==num]['speech'].head(1))

In [ ]:
bert_sp_List = set()
lda_sp_List = set()
nmf_sp_List = set()
check_id = 0

for i in range(len(fullData)):
    check_id=check_id+1
    id=fullData['speech_id'][i]
    for bkw in bert_kw_List:
        if fullData['Topic'][i] == bkw:
            bert_sp_List.add(id)
    for lkw in lda_kw_List:
        if fullData['Most_Dominant_Topic_LDA'][i] == lkw:
            lda_sp_List.add(id)
    for nkw in nmf_kw_List:
        if fullData['Most_Dominant_Topic_NMF'][i] == nkw:
            nmf_sp_List.add(id)

#bert_sp_List = set(bert_sp_List)
#lda_sp_List = set(lda_sp_List)
#nmf_sp_List = set(nmf_sp_List)

collocated words for top keywords
before and after 2008 and 2001
    some readings
check out the pattern/ trend for minor topic ( without topic 3)
sentimental analysis
    tidytext


In [2]:
bert_kw_List = [3,2761, 4476, 2343, 7408, 43,115,595,672,1311,2643,2805,2855,3184,4148,4690,4821,4887,5892,6154,6159,6532,6675,6820]
#df = pd.read_csv('gpt4_5000_result.csv')


bert_sp_List = set()
check_id = 0

for i in range(len(fullData)):
    id=fullData['speech_id'][i]
    for bkw in bert_kw_List:
        if fullData['Topic'][i] == bkw:
            check_id=check_id+1
            bert_sp_List.add(id)

#bert_sp_List = set(bert_sp_List)
print(check_id)
common_speech_ids = set(df['speech_id']).intersection(bert_sp_List)
print("yes")
print(len(common_speech_ids))


# Convert set to a Pandas DataFrame
#df = pd.DataFrame({'set_column': [bert_sp_List]})

# Specify the CSV file path
#csv_file_path = 'random50EnvironmentSpeechid.csv'

# Write to CSV file
#df.to_csv(csv_file_path, index=False)



7810
yes
72


In [19]:
# save whatever into pickle
pickle_file_path = 'bert_environ.pkl'

# Open the pickle file for writing in binary mode ('wb')
with open(pickle_file_path, 'wb') as file:
    # Dump the set into the pickle file
    pickle.dump(common_speech_ids, file)

In [3]:
# read Louis pickle and take out mannual annotated "yes"
import pickle

# Specify the path to your pickle file
pickle_file_path = 'manualAnnotation.pkl'

# Open the pickle file for reading in binary mode ('rb')
with open(pickle_file_path, 'rb') as file:
    # Load the data from the pickle file
    result35 = pickle.load(file)
# mannual annotate
keys_with_y_value = {key for key, value in result35.items() if value == 'y'}

In [18]:
# compare set of "yes" from various models
#87 vs 72: 22 in common
common_speech_ids_bert_35 = keys_with_y_value.intersection(common_speech_ids)
print(len(common_speech_ids_bert_35))

22


In [15]:
# Convert set to Pandas DataFrame
import random
sampled_items = random.sample(sorted(bert_sp_List), 100)

df = pd.DataFrame({'set_column': [sampled_items]})

# Explode the set into separate rows
df_exploded = df['set_column'].explode().reset_index(drop=True).to_frame()

# Specify the CSV file path
csv_file_path = 'random50EnvironmentSpeechid_100.csv'

# Write to CSV file
df_exploded.to_csv(csv_file_path, index=False, header=['set_column'])

In [25]:
# cross-check environment topic from bert
fullData = pd.read_csv('test50_labled.csv')
bert_kw_List = [3,2761, 4476, 2343, 4294, 7408, 43,110,115,595,672,849,1311,2643,2805,2855,3184,3878,4148,4690,4821,4887,5892,6154,6159,6228,6532,6675,6820]

filtered_data = fullData[
    fullData.apply(lambda row: row['Representative_document'] and row['Topic'] in bert_kw_List, axis=1)
]

In [33]:
# open json for bertopic keyword and topic dictionary
import json
# Specify the path to your JSON file
json_file_path = 'Bertopic_50_dic.json'

# Open the JSON file for reading
with open(json_file_path, 'r') as file:
    # Load the JSON data from the file
    bertKeyWordic = json.load(file)


#### Note about Bertopic result
bert_kw_List = [3,2761, 4476, 2343, 7408, 43,115,595,672,1311,2643,2805,2855,3184,4148,4690,4821,4887,5892,6154,6159,6532,6675,6820]

previously wrong from standford keyword: 110( the word "object"), 4294('na'), 6228('include location Penn'), 849( include California), 3878(texas)

In [ ]:
bert_kw_List = [3,2761, 4476, 2343, 7408, 43,115,595,672,1311,2643,2805,2855,3184,4148,4690,4821,4887,5892,6154,6159,6532,6675,6820]
filtered_data_only_correct_en = fullData[
    fullData.apply(lambda row: row['Topic'] in bert_kw_List, axis=1)
]
filtered_data_only_correct_en.to_csv("Environment")

In [ ]:
[['speech','chamber','date','speaker','first_name','last_name','state','gender','char_count','word_count']]

In [ ]:
fullData

In [43]:
# all demographic information for speeches
# with full demo data
fullData = pd.read_csv('fullData2000.csv')
# join topic and keywords
lables = pd.read_csv('test50_labled.csv')
label_fullSpeech_demo = pd.merge(lables[['Topic', 'Representation','Representative_document','speech_id']], 
                                 fullData[['speech_id','speech','chamber','date','speaker','first_name','last_name','state','gender','char_count','word_count']],
                                 on='speech_id', how='left')

# filter and only keep environment relevant topics
bert_kw_List = [3,2761, 4476, 2343, 7408, 43,115,595,672,1311,2643,2805,2855,3184,4148,4690,4821,4887,5892,6154,6159,6532,6675,6820]
filtered_data_demo = label_fullSpeech_demo[label_fullSpeech_demo.apply(lambda row: row['Topic'] in bert_kw_List, axis=1)]
#save to csv
filtered_data_demo.to_csv("EnvironmentSpeech_WithInfo.csv")

In [ ]:
pd.set_option('display.max_colwidth', None)

#import pandas as pd
#filtered_data_demo=lables = pd.read_csv('subsample.csv')
filtered_data_demo[filtered_data_demo.apply(lambda row: row['Topic'] == 3, axis =1)]['Representation']

#### Co-located
with specific words:</p>
https://stackoverflow.com/questions/21165702/nltk-collocations-for-specific-words

fullData = pd.read_csv('fullData2000.csv')


In [1]:
import pandas as pd

from nltk import word_tokenize, bigrams, trigrams
from nltk.corpus import stopwords
from collections import Counter
#nltk.download('punkt')

fullData = pd.read_csv('EnvironmentSpeech_WithInfo.csv')
topicList = sorted(fullData['Topic'].unique())
#result_df = pd.DataFrame()

In [18]:
fullData

,Unnamed: 0,Topic,Representation,Representative_document,speech_id,speech,chamber,date,speaker,first_name,last_name,state,gender,char_count,word_count,tokens,filtered_tokens,bigrams,trigrams
0,184,115,"['forest', 'forests', 'timber', 'fires', 'tree...",False,1060076829,Mr. President. this amendment would strike sec...,S,1999-09-09,Mr. ROBB,Unknown,ROBB,Unknown,M,1355,218,"[Mr., President, ., this, amendment, would, st...","[amendment, strike, section, legislative, ride...","[(amendment, strike), (strike, section), (sect...","[(amendment, strike, section), (strike, sectio..."
1,302,2343,"['earth', 'gaylord', 'nelson', 'environmental'...",False,1060057330,I thank the gentleman for yielding. It just st...,H,1999-07-13,Mr. REGULA,Unknown,REGULA,Unknown,M,322,57,"[I, thank, the, gentleman, for, yielding, ., I...","[yielding, struck, visited, mount, helens, las...","[(yielding, struck), (struck, visited), (visit...","[(yielding, struck, visited), (struck, visited..."
2,616,43,"['wilderness', 'park', 'lands', 'monument', 'l...",False,1060007645,Mr. Speaker. today I come before the House of ...,E,1999-03-02,Ms. DUNN,Unknown,DUNN,Unknown,F,1246,214,"[Mr., Speaker, ., today, I, come, before, the,...","[house, representatives, wish, ahappy, birthda...","[(house, representatives), (representatives, w...","[(house, representatives, wish), (representati..."
3,649,3,"['oil', 'gas', 'energy', 'drilling', 'gasoline...",False,1060017006,Mr. President. we are ten years older. but are...,S,1999-03-24,Mr. MURKOWSKI,Unknown,MURKOWSKI,Unknown,M,7558,1271,"[Mr., President, ., we, are, ten, years, older...","[ten, years, older, ten, years, wiser, since, ...","[(ten, years), (years, older), (older, ten), (...","[(ten, years, older), (years, older, ten), (ol..."
4,666,672,"['river', 'dams', 'dam', 'missouri', 'downstre...",False,1060025320,cochair the 16 member upper Mississippi River ...,H,1999-04-29,Mr. LEACH,Unknown,LEACH,Unknown,M,1304,207,"[cochair, the, 16, member, upper, Mississippi,...","[cochair, member, upper, mississippi, river, c...","[(cochair, member), (member, upper), (upper, m...","[(cochair, member, upper), (member, upper, mis..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7080,763203,3,"['oil', 'gas', 'energy', 'drilling', 'gasoline...",False,1140086554,for his leadership on this and for coming to j...,H,2016-04-19,Mr. KILMER,Unknown,KILMER,Unknown,M,3340,535,"[for, his, leadership, on, this, and, for, com...","[leadership, coming, join, us, speaking, clima...","[(leadership, coming), (coming, join), (join, ...","[(leadership, coming, join), (coming, join, us..."
7081,763831,43,"['wilderness', 'park', 'lands', 'monument', 'l...",False,1140102037,Mr. Speaker. I rise today to honor Mr. Kenneth...,E,2016-06-16,Mr. THOMPSON of California,Unknown,THOMPSON,California,M,2335,363,"[Mr., Speaker, ., I, rise, today, to, honor, M...","[rise, honor, kenneth, hofmann, owner, rancho,...","[(rise, honor), (honor, kenneth), (kenneth, ho...","[(rise, honor, kenneth), (honor, kenneth, hofm..."
7082,763834,6675,"['tree', 'arbor', 'fresno', 'trees', 'usa', 'r...",True,1140104694,Mr. Speaker. I rise today to recognize the Cit...,E,2016-07-05,Mr. YOUNG of Iowa,Unknown,YOUNG,Iowa,M,1327,210,"[Mr., Speaker, ., I, rise, today, to, recogniz...","[rise, recognize, city, west, des, moines, iow...","[(rise, recognize), (recognize, city), (city, ...","[(rise, recognize, city), (recognize, city, we..."
7083,763963,3,"['oil', 'gas', 'energy', 'drilling', 'gasoline...",False,1140085590,Mr. President. I agree with the Republican lea...,S,2016-04-14,Mr. REID,Unknown,REID,Unknown,M,1728,341,"[Mr., President, ., I, agree, with, the, Repub...","[agree, republican, leader, energy, bill, good...","[(agree, republican), (republican, leader), (l...","[(agree, republican, leader), (republican, lea..."


In [2]:
# Check keywords for each topics
pd.set_option('display.max_colwidth', None)
keywordList = fullData[fullData.apply(lambda row: row['Topic'] == topicList[0], axis =1)].head(1)['Representation']
keywordList

3    ['oil', 'gas', 'energy', 'drilling', 'gasoline', 'prices', 'renewable', 'drill', 'wind', 'barrels']
Name: Representation, dtype: object

In [3]:
def colocated(target_word, fullData,result):
    fullData['tokens'] = fullData['speech'].apply(word_tokenize)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    newStop=['president','object','another','point','get','must','made','two','hr','say','go','come','much','may','make','even','going','also','would','could','think','chair','mr','madam','dont','said','did','does','thank','ask','today','just','000','gentleman','speaker','floor']
    for word in newStop:
        stop_words.add(word)
    fullData['filtered_tokens'] = fullData['tokens'].apply(lambda x: [word.lower() for word in x if word.isalpha() and word.lower() not in stop_words])

    # Find bigrams and trigrams
    fullData['bigrams'] = fullData['filtered_tokens'].apply(lambda x: list(bigrams(x)))
    fullData['trigrams'] = fullData['filtered_tokens'].apply(lambda x: list(trigrams(x)))

    # Flatten the list of bigrams and trigrams
    all_bigrams = [item for sublist in fullData['bigrams'] for item in sublist]
    all_trigrams = [item for sublist in fullData['trigrams'] for item in sublist]

    # Count occurrences
    bigram_counts = Counter(all_bigrams)
    trigram_counts = Counter(all_trigrams)

    # most common pairs overall
    #co_occurring_bigrams = bigram_counts.most_common(15)
    #co_occurring_trigrams = trigram_counts.most_common(15)

    # Get the bigrams and trigrams that co-occur with the target word in FULL
    co_occurring_bigrams = [bigram for bigram in bigram_counts if target_word in bigram][:10]
    co_occurring_trigrams = [trigram for trigram in trigram_counts if target_word in trigram][:10]
    
    # Store the co-located words (excluding the target word) in lists
    co_located_bigram_words = []
    for bigram in co_occurring_bigrams:
        co_located_bigram_words.extend([word for word in bigram if word != target_word])
    
    co_located_trigram_words = []
    for trigram in co_occurring_trigrams:
        co_located_trigram_words.extend([word for word in trigram if word != target_word])

    # Print the stored lists
    print("Co-located Bigram Words without '{}':".format(target_word), co_located_bigram_words)
    #print("Co-located Trigram Words without '{}':".format(target_word), co_occurring_trigrams)
  
    
    #result_df[target_word] = co_located_bigram_words


In [4]:
# find trigram for specific bigram
target_word_1 = 'energy'
target_word_2 = 'heat'

fullData['tokens'] = fullData['speech'].apply(word_tokenize)
stop_words = set(stopwords.words('english'))
newStop=['president','object','another','point','get','must','made','two','hr','say','go','come','much','may','make','even','going','also','would','could','think','chair','mr','madam','dont','said','did','does','thank','ask','today','just','000','gentleman','speaker','floor']
for word in newStop:
    stop_words.add(word)
fullData['filtered_tokens'] = fullData['tokens'].apply(lambda x: [word.lower() for word in x if word.isalpha() and word.lower() not in stop_words])

fullData['trigrams'] = fullData['filtered_tokens'].apply(lambda x: list(trigrams(x)))
all_trigrams = [item for sublist in fullData['trigrams'] for item in sublist]

trigram_counts = Counter(all_trigrams)
co_located_trigrams = [trigram for trigram, count in trigram_counts.items() if target_word_1 in trigram and target_word_2 in trigram]
print(co_located_trigrams)

#fullData['bigrams'] = fullData['filtered_tokens'].apply(lambda x: list(bigrams(x)))
#all_bigrams = [item for sublist in fullData['bigrams'] for item in sublist]
#bigram_counts = Counter(all_bigrams)
#co_located_bigrams = [bigram for bigram, count in bigram_counts.items() if target_word_1 in bigram and target_word_2 in bigram]

[('produce', 'heat', 'energy'), ('heat', 'energy', 'electrical'), ('mechanical', 'energy', 'heat'), ('energy', 'heat', 'energy'), ('heat', 'energy', 'led'), ('heat', 'pumps', 'energy'), ('energy', 'resourcesthe', 'heat'), ('turned', 'heat', 'energy'), ('heat', 'energy', 'largely'), ('unusable', 'heat', 'energy'), ('heat', 'energy', 'eventually'), ('unless', 'heat', 'energy'), ('heat', 'energy', 'smell'), ('geothermal', 'energy', 'heat'), ('heat', 'energy', 'within'), ('energy', 'assistance', 'heat'), ('energy', 'people', 'heat'), ('energy', 'goes', 'heat'), ('affordable', 'energy', 'heat'), ('energy', 'heat', 'homes'), ('lot', 'energy', 'heat'), ('energy', 'heat', 'people'), ('live', 'heat', 'energy'), ('heat', 'energy', 'comes'), ('country', 'energy', 'heat'), ('energy', 'heat', 'business'), ('heat', 'business', 'energy'), ('sure', 'energy', 'heat'), ('form', 'energy', 'heat'), ('energy', 'heat', 'gets'), ('heat', 'homes', 'energy'), ('energy', 'used', 'heat'), ('energy', 'generate', 

In [54]:
# find frequency for targeted trigram
print(trigram_counts[('energy', 'economy', 'breathe')])
print(co_located_trigrams)


1
[('energy', 'economy', 'breathe')]


In [ ]:
# present speech for specific ID
pd.set_option('display.max_colwidth', None)
fullData[fullData['speech_id']==1070120577]['speech']

In [ ]:
# find trigram ID and row
speeches_with_trigrams = fullData[fullData['trigrams'].apply(lambda x: any(trigram in x for trigram in co_located_trigrams))]
speeches_with_trigrams

In [32]:
result_df =1
colocated('renewable',fullData,result_df)

Co-located Bigram Words without 'renewable': ['trees', 'resource', 'timber', 'situation', 'clean', 'energy', 'winwin', 'technologies', 'many', 'applications']
Co-located Trigram Words without 'renewable': [('timber', 'trees', 'renewable'), ('trees', 'renewable', 'resource'), ('renewable', 'resource', 'harvested'), ('reality', 'timber', 'renewable'), ('timber', 'renewable', 'resource'), ('renewable', 'resource', 'properly'), ('ethanol', 'situation', 'renewable'), ('situation', 'renewable', 'clean'), ('renewable', 'clean', 'still'), ('renewable', 'energy', 'winwin')]


In [79]:
result_df

,oil,energy,renewable,wilderness
0,valdez,alternative,trees,nepa
1,spill,sources,resource,act
2,worst,qualified,timber,designation
3,crude,producing,situation,nearby
4,bubbled,isolated,clean,protect
5,provision,grids,energy,values
6,pollution,systems,winwin,consider
7,aspects,take,technologies,designations
8,transport,new,many,new
9,alaskas,needs,applications,peaks


In [ ]:
#target_word = 'oil','energy','renewable','wilderness'
target_list =['park','forest','timber','lake','freshwater','river','dam']
for word in target_list:
    target_word = word
    colocated(target_word,fullData,result_df)

### Zoom into 2008

In [5]:
import pandas as pd
fullData = pd.read_csv('fullData2000.csv')
#2008 only 
fullData['date'] = pd.to_datetime(fullData['date'])
data_2008 = fullData[fullData['date'].dt.year == 2008]

In [ ]:
data = data_2008
# index must be reset if the data is subsampled
data.reset_index(drop=True, inplace=True)

# you might want to use seperate environment for Bert and sklearn, 
# as they have different version requirement for some common dependencies
from bertopic import BERTopic

# train bert and handle overtime changes
bert_model = BERTopic(verbose=True)
topics, _ = bert_model.fit_transform(data['speech'])
topics_over_time = bert_model.topics_over_time(data['speech'], data['date'], nr_bins=12)
#get some visualization
bert_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [26]:
import pickle
# Save the BERTopic model
with open('bert_model.pkl', 'wb') as model_file:
    pickle.dump(bert_model, model_file)

# Save the topics_over_time result
with open('topics_over_time.pkl', 'wb') as topics_file:
    pickle.dump(topics_over_time, topics_file)

c:\Users\tanay\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [25]:
bert_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [24]:
data.reset_index(drop=True, inplace=True)
topics_over_time = bert_model.topics_over_time(data['speech'], data['date'], nr_bins=12)

12it [02:26, 12.20s/it]


In [1]:
import pickle
# FOR FUTURE TO LOAD
# Load the BERTopic model
with open('bert_model.pkl', 'rb') as model_file:
    loaded_bert_model = pickle.load(model_file)

# Load the topics_over_time result
with open('topics_over_time.pkl', 'rb') as topics_file:
    loaded_topics_over_time = pickle.load(topics_file)

c:\Users\tanay\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
wantET = loaded_bert_model.find_topics(search_term='pollution')
wantET

([1346, 62, 379, 449, 519],
 [0.3908963, 0.3604297, 0.35902733, 0.3588, 0.3348549])

In [38]:
loaded_bert_model.get_topic(185)

[('liheap', 0.015207976516313013),
 ('heating', 0.010615511776768382),
 ('winter', 0.00983789981307507),
 ('heat', 0.006004608152990103),
 ('lowincome', 0.004165787083347811),
 ('assistance', 0.003985760317357853),
 ('warm', 0.003596089666172708),
 ('cool', 0.003393919982428345),
 ('funding', 0.003257120174924511),
 ('vermont', 0.0030733618114999355)]

In [36]:
from bertopic import BERTopic
# Subset your topics_over_time dataframe
selected_topics_df = loaded_topics_over_time[loaded_topics_over_time.Topic.isin([752, 36,62,131,72,1166,379,508,1346,45])]

# Visualize your selected topics
loaded_bert_model.visualize_topics_over_time(selected_topics_df)

#### stop-words or no?: BERTopic
https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html#removing-stop-words
https://github.com/MaartenGr/BERTopic/issues/40